In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/airqo-air-quality-prediction-challenge-zindi/SampleSubmission (9).csv
/kaggle/input/airqo-air-quality-prediction-challenge-zindi/Test (9).csv
/kaggle/input/airqo-air-quality-prediction-challenge-zindi/Train (18).csv


In [2]:
train = pd.read_csv('/kaggle/input/airqo-air-quality-prediction-challenge-zindi/Train (18).csv')
test = pd.read_csv('/kaggle/input/airqo-air-quality-prediction-challenge-zindi/Test (9).csv')
sub = pd.read_csv('/kaggle/input/airqo-air-quality-prediction-challenge-zindi/SampleSubmission (9).csv')

In [3]:
all_data = pd.concat([train, test])

In [4]:
all_data['date'] = pd.to_datetime(all_data['date'])

all_data['Year'] = all_data['date'].dt.year
all_data['Day'] = all_data['date'].dt.day
all_data['Day_of_Week'] = all_data['date'].dt.dayofweek

In [5]:
# import pandas as pd
# import numpy as np
import datetime

class Preprocessor:
    def __init__(self, df):
        self.df = df
        self.add_place_id()
    
    def add_place_id(self, preserve_place_str=False):
        coordinates = [(row['site_latitude'], row['site_longitude']) for _, row in self.df[['site_latitude', 'site_longitude']].drop_duplicates().iterrows()]
        coords = {coord: idx for idx, coord in enumerate(coordinates)}
        
        self.df['pid'] = self.df.apply(lambda row: coords[(row['site_latitude'], row['site_longitude'])], axis=1)
        self.coords = np.array([tuple(row) for row in coordinates])
        
        if not preserve_place_str:
            self.df.drop(['site_latitude', 'site_longitude'], axis=1, inplace=True)


In [6]:
preprocessor = Preprocessor(all_data)
processed_df = preprocessor.df
processed_df

,id,site_id,city,country,date,hour,sulphurdioxide_so2_column_number_density,sulphurdioxide_so2_column_number_density_amf,sulphurdioxide_so2_slant_column_number_density,sulphurdioxide_cloud_fraction,...,cloud_surface_albedo,cloud_sensor_azimuth_angle,cloud_sensor_zenith_angle,cloud_solar_azimuth_angle,cloud_solar_zenith_angle,pm2_5,Year,Day,Day_of_Week,pid
0,id_vjcx08sz91,6531a46a89b3300013914a36,Lagos,Nigeria,2023-10-25,13,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,12.015000,2023,25,2,0
1,id_bkg215syli,6531a46a89b3300013914a36,Lagos,Nigeria,2023-11-02,12,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,42.267200,2023,2,3,0
2,id_oui2pot3qd,6531a46a89b3300013914a36,Lagos,Nigeria,2023-11-03,13,NaN,NaN,NaN,NaN,...,0.192757,-96.411890,61.045123,-121.307414,41.898269,39.450741,2023,3,4,0
3,id_9aandqzy4n,6531a46a89b3300013914a36,Lagos,Nigeria,2023-11-08,14,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,10.537600,2023,8,2,0
4,id_ali5x2m4iw,6531a46a89b3300013914a36,Lagos,Nigeria,2023-11-09,13,0.000267,0.774656,0.000207,0.223403,...,0.153114,-97.811241,49.513439,-126.064453,40.167355,19.431731,2023,9,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2778,id_5yblexf7zp,64654b4dd5320e001d6c499a,Kisumu,Kenya,2024-01-27,10,0.000099,0.660385,0.000066,0.209272,...,0.250637,72.428178,55.404121,-150.645025,21.248325,NaN,2024,27,5,106
2779,id_c9cycuvetl,65c8c557b3d86f0012b2e32b,Yaoundé,Cameroon,2024-02-21,12,NaN,NaN,NaN,NaN,...,0.228072,-101.384688,16.577282,-125.511751,25.086502,NaN,2024,21,2,107
2780,id_8xu1ardxni,65c8c557b3d86f0012b2e32b,Yaoundé,Cameroon,2024-02-22,12,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2024,22,3,107
2781,id_aumy97t1iu,65c8c557b3d86f0012b2e32b,Yaoundé,Cameroon,2024-02-23,12,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2024,23,4,107


In [7]:
processed_df

,id,site_id,city,country,date,hour,sulphurdioxide_so2_column_number_density,sulphurdioxide_so2_column_number_density_amf,sulphurdioxide_so2_slant_column_number_density,sulphurdioxide_cloud_fraction,...,cloud_surface_albedo,cloud_sensor_azimuth_angle,cloud_sensor_zenith_angle,cloud_solar_azimuth_angle,cloud_solar_zenith_angle,pm2_5,Year,Day,Day_of_Week,pid
0,id_vjcx08sz91,6531a46a89b3300013914a36,Lagos,Nigeria,2023-10-25,13,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,12.015000,2023,25,2,0
1,id_bkg215syli,6531a46a89b3300013914a36,Lagos,Nigeria,2023-11-02,12,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,42.267200,2023,2,3,0
2,id_oui2pot3qd,6531a46a89b3300013914a36,Lagos,Nigeria,2023-11-03,13,NaN,NaN,NaN,NaN,...,0.192757,-96.411890,61.045123,-121.307414,41.898269,39.450741,2023,3,4,0
3,id_9aandqzy4n,6531a46a89b3300013914a36,Lagos,Nigeria,2023-11-08,14,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,10.537600,2023,8,2,0
4,id_ali5x2m4iw,6531a46a89b3300013914a36,Lagos,Nigeria,2023-11-09,13,0.000267,0.774656,0.000207,0.223403,...,0.153114,-97.811241,49.513439,-126.064453,40.167355,19.431731,2023,9,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2778,id_5yblexf7zp,64654b4dd5320e001d6c499a,Kisumu,Kenya,2024-01-27,10,0.000099,0.660385,0.000066,0.209272,...,0.250637,72.428178,55.404121,-150.645025,21.248325,NaN,2024,27,5,106
2779,id_c9cycuvetl,65c8c557b3d86f0012b2e32b,Yaoundé,Cameroon,2024-02-21,12,NaN,NaN,NaN,NaN,...,0.228072,-101.384688,16.577282,-125.511751,25.086502,NaN,2024,21,2,107
2780,id_8xu1ardxni,65c8c557b3d86f0012b2e32b,Yaoundé,Cameroon,2024-02-22,12,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2024,22,3,107
2781,id_aumy97t1iu,65c8c557b3d86f0012b2e32b,Yaoundé,Cameroon,2024-02-23,12,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2024,23,4,107


In [8]:
all_data_2 = all_data.drop(columns = ['id', 'pm2_5', 'city', 'site_id', 'country'])

In [9]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

cat = all_data_2.columns[all_data_2.dtypes == 'object']

In [10]:
for category in cat:
    
    all_data_2[category] = le.fit_transform(all_data_2[category])

In [11]:
train_2 = all_data_2[:len(train)]
test_2 = all_data_2[len(train):]

In [12]:
from catboost import CatBoostRegressor

In [20]:
from sklearn.model_selection import GroupKFold

GKF = GroupKFold(n_splits = 10)

result = 0

for train_index, val_index in GKF.split(train_2, groups = train['site_id']):
    
    x_train, x_val = train_2.iloc[train_index], train_2.iloc[val_index]
    
    y_train, y_val = train['pm2_5'].iloc[train_index], train['pm2_5'].iloc[val_index]
    
    cbr = CatBoostRegressor(verbose = 500, iterations = 5000, learning_rate = 0.03)
    
    cbr.fit(x_train, np.log(y_train), eval_set = (x_val, np.log(y_val)), early_stopping_rounds = 50, cat_features = ['pid'])
    
    temp = np.exp(cbr.predict(test_2))
    
    result += temp / 10 

0:	learn: 0.6272398	test: 0.7058996	best: 0.7058996 (0)	total: 19.4ms	remaining: 1m 37s
500:	learn: 0.3352863	test: 0.5229444	best: 0.5220221 (452)	total: 6.3s	remaining: 56.6s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.5220221322
bestIteration = 452

Shrink model to first 453 iterations.
0:	learn: 0.6344530	test: 0.6448796	best: 0.6448796 (0)	total: 17.3ms	remaining: 1m 26s
500:	learn: 0.3268344	test: 0.5258597	best: 0.5257866 (499)	total: 5.74s	remaining: 51.5s
1000:	learn: 0.2807612	test: 0.5056514	best: 0.5056460 (999)	total: 11.3s	remaining: 45.1s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.5035770927
bestIteration = 1116

Shrink model to first 1117 iterations.
0:	learn: 0.6295215	test: 0.6843478	best: 0.6843478 (0)	total: 15.6ms	remaining: 1m 18s
500:	learn: 0.3221553	test: 0.5393242	best: 0.5385325 (486)	total: 5.6s	remaining: 50.3s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.5376587937
bestIteration = 548


In [21]:
result

array([15.52692792, 18.26154766, 13.34530732, ..., 40.74331124,
       38.09695874, 33.48274865])

In [22]:
sub['pm2_5'] = result

In [23]:
sub

,id,pm2_5
0,id_ihxgrbq8bw,15.526928
1,id_dg6s4fhiwe,18.261548
2,id_f7hwwtmuzp,13.345307
3,id_ioese5awdg,16.203736
4,id_hdw320zpls,14.748721
...,...,...
2778,id_5yblexf7zp,26.983300
2779,id_c9cycuvetl,33.386516
2780,id_8xu1ardxni,40.743311
2781,id_aumy97t1iu,38.096959


In [24]:
sub.to_csv('airquality_zindi.csv', index = False)